<a href="https://colab.research.google.com/github/jhno-code/2020-2/blob/main/Funciones_Basicas_Mpi__hasta_send_y_recv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributed Parallel Programming Patterns using mpi4py
Libby Shoop, Macalester College

¡Bienvenido!

Este libro contiene algunos ejemplos que ilustran los conceptos fundamentales básicos de la computación distribuida utilizando código Python. El tipo de computación que ilustran estos ejemplos se llama * paso de mensajes *. El paso de mensajes es una forma de programación que se basa en procesos que se comunican entre sí para coordinar su trabajo. El paso de mensajes se puede utilizar en una sola computadora multinúcleo o con un grupo de computadoras.


### Patrones de software

Los patrones en el software son implementaciones comunes que los profesionales han utilizado una y otra vez para realizar tareas. A medida que los profesionales los usan repetidamente, la comunidad comienza a darles nombres y catalogarlos, convirtiéndolos a menudo en funciones de biblioteca reutilizables. Los ejemplos que verá en este libro se basan en patrones documentados que se han utilizado para resolver diferentes problemas mediante el paso de mensajes entre procesos. El paso de mensajes es una forma de computación distribuida que usa procesos, que se pueden usar en grupos de computadoras o máquinas multinúcleo.

En muchos de estos ejemplos, el nombre del patrón es parte del nombre del archivo de código Python. También verá que, a menudo, las funciones de la biblioteca MPI también toman el nombre del patrón, y la implementación de esas funciones contiene el patrón que los profesionales utilizan con frecuencia. Estos ejemplos de código de patrón que le mostramos aquí, denominados patternlets, se basan en el trabajo original de Joel Adams:

Adams, Joel C. "Patternlets: una herramienta de enseñanza para presentar a los estudiantes los patrones de diseño paralelos". Taller del Simposio Internacional de Procesamiento Distribuido y Paralelo del IEEE 2015. IEEE, 2015.

Para ejecutar estos ejemplos, primero necesitará instalar la biblioteca mpi4py ejecutando este código (esto generalmente llevará un tiempo instalarlo la primera vez):

In [ ]:
! pip install mpi4py

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s)**Important:** 
Importante: tendrá que volver a ejecutar esta celda después de desconectarse durante bastante tiempo.

# Patrones de estructura del programa

> Bloque con sangría



## Single Program, Multiple Data SPMD

1.   Elemento de lista
2.   Elemento de lista



This code forms the basis of all of the other examples that follow. It is the fundamental way we structure parallel programs today.


In [ ]:
%%writefile ejercicio1.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    
    print("Saludos desde el proceso {} de {} "\
    .format(id, numProcesses))

########## Run the main function
main()


Overwriting ejercicio1.py


In [ ]:
! mpirun --allow-run-as-root -np 5 python ejercicio1.py

Saludos desde el proceso 0 de 5 
Saludos desde el proceso 1 de 5 
Saludos desde el proceso 4 de 5 
Saludos desde el proceso 3 de 5 
Saludos desde el proceso 2 de 5 


The fundamental idea of message passing programs can be illustrated like this:

![picture](https://drive.google.com/uc?id=1wpQaFiaubIcQBV9Lw_jwOU0y2-K-EChW)

Cada proceso se configura dentro de una red de comunicación para poder comunicarse con todos los demás procesos a través de enlaces de comunicación. Cada proceso está configurado para tener su propio número, o id, que comienza en 0.

Nota: Cada proceso tiene sus propias copias de las 4 variables de datos anteriores. Entonces, aunque hay un solo programa, se ejecuta varias veces en procesos separados, cada uno con sus propios valores de datos. Esta es la razón del nombre del patrón que representa este código: programa único, datos múltiples. La línea de impresión al final de main () representa las múltiples salidas de datos diferentes producidas por cada proceso.


## Master-Worker
Este también es un patrón muy común que se usa en programación paralela y distribuida. Aquí está el pequeño código ilustrativo de muestra. Revísalo y responde esto: ¿Qué es diferente entre este ejemplo y el anterior?

In [ ]:
%%writefile 01masterWorker.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if id == 0:
        print("Saludos del maestro, {} de {} en {}"\
        .format(id, numProcesses, myHostName))
    else:
        print("Saludos del trabajador, {} de {} en {}"\
        .format(id, numProcesses, myHostName))

########## Run the main function
main()


Writing 01masterWorker.py


La respuesta a la pregunta anterior ilustra lo que podemos hacer con este patrón: basándonos en la identificación del proceso, podemos hacer que un proceso lleve a cabo algo diferente a los demás. Este concepto se utiliza mucho como un medio para coordinar actividades, donde un proceso, a menudo llamado maestro, tiene la responsabilidad de entregar el trabajo y realizar un seguimiento de los resultados. Veremos esto en ejemplos posteriores.

**Note:**
Por convención, el proceso de coordinación maestro suele ser el proceso número 0.

In [ ]:
! mpirun --allow-run-as-root -np 6 python 01masterWorker.py

Saludos del trabajador, 1 de 6 en 4ea9d432a738
Saludos del trabajador, 4 de 6 en 4ea9d432a738
Saludos del maestro, 0 de 6 en 4ea9d432a738
Saludos del trabajador, 2 de 6 en 4ea9d432a738
Saludos del trabajador, 3 de 6 en 4ea9d432a738
Saludos del trabajador, 5 de 6 en 4ea9d432a738


### Exercises:

- Vuelva a ejecutar, utilizando un número variable de procesos del 1 al 8 (es decir, varíe el argumento después de -np).

- Explique qué permanece igual y qué cambia a medida que cambia el número de procesos.

# Descomposición usando paralelismo para patrones de bucles

La forma más común de completar una tarea repetida en cualquier lenguaje de programa es un bucle. Usamos bucles porque queremos hacer un cierto número de tareas, muy a menudo porque queremos trabajar en un conjunto de elementos de datos que se encuentran en una lista o una matriz, o alguna otra estructura de datos. Si el trabajo a realizar en cada ciclo es independiente de las iteraciones anteriores, podemos usar procesos separados para hacer partes del ciclo de forma independiente. Este patrón de estructura de programa se denomina patrón de bucle para paralelizar, que es una estrategia de implementación para la descomposición del trabajo a realizar en partes más pequeñas.

## Parallel Loop Split into Equal Sized Chunks -> G1

In the code below, notice the use of the variable called `REPS`. This is designed to be the total amount or work, or repetitions, that the for loop is accomplishing. This particular code is designed so that if those repetitions do not divide equally by the number of processes, then the program will stop with a warning message printed by the master process.

Remember that because this is still also a SPMD program, all processes execute the code in the part of the if statement that evaluates to True. Each process has its own id, and we can determine how many processes there are, so we can choose where in the overall number of REPs of the loop each process will execute.

En el código siguiente, observe el uso de la variable llamada "REPS". Esto está diseñado para ser la cantidad total de trabajo, o repeticiones, que el ciclo for está logrando. Este código en particular está diseñado para que si esas repeticiones no se dividen equitativamente por el número de procesos, el programa se detendrá con un mensaje de advertencia impreso por el proceso maestro.

Recuerde que debido a que también es un programa SPMD, todos los procesos ejecutan el código en la parte de la instrucción if que se evalúa como Verdadero. Cada proceso tiene su propia identificación, y podemos determinar cuántos procesos hay, por lo que podemos elegir en qué parte del número total de REP del ciclo se ejecutará cada proceso.

In [ ]:
%%writefile 02parallelLoopEqualChunks.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 8

    if ((REPS % numProcesses) == 0 and numProcesses <= REPS):
        # How much of the loop should a process work on?
        chunkSize = int(REPS / numProcesses) #1
        start = id * chunkSize 
        stop = start + chunkSize

        # do the work within the range set aside for this process
        for i in range(start, stop): # 0 a 4
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i))
    else:
        # cannot break into equal chunks; one process reports the error
        if id == 0 :
            print("Please run with number of processes divisible by \
and less than or equal to {}.".format(REPS))

########## Run the main function
main()


Overwriting 02parallelLoopEqualChunks.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 02parallelLoopEqualChunks.py

On aa7fe3e77f51: Process 1 is performing iteration 2
On aa7fe3e77f51: Process 2 is performing iteration 4
On aa7fe3e77f51: Process 1 is performing iteration 3
On aa7fe3e77f51: Process 0 is performing iteration 0
On aa7fe3e77f51: Process 3 is performing iteration 6
On aa7fe3e77f51: Process 2 is performing iteration 5
On aa7fe3e77f51: Process 0 is performing iteration 1
On aa7fe3e77f51: Process 3 is performing iteration 7


### Exercises

- Run, using these numbers of processes, N: 1, 2, 4, and 8 (i.e., vary the  argument to -np).
- Change REPS to 16 in the code and rerun it. Then rerun with mpirun, varying N again.
- Explain how this pattern divides the iterations of the loop among the processes.

Which of the following is the correct assignment of loop iterations to processes for this code, when REPS is 8 and numProcesses is 4?


![picture](https://drive.google.com/uc?id=1eUsjxYdWXWqThO_rdLO91HaLqBLAAh_S)

## Parallel for Loop Program Structure: chunks of 1

In the code below, we again use the variable called `REPS` for the total amount or work, or repetitions, that the for loop is accomplishing. This particular code is designed so that the number of repetitions should be more than or equal to the number of processes requested.
.. note:: Typically in real problems, the number of repetitions is much higher than the number of processes. We keep it small here to illustrate what is happening.

Like the last example all processes execute the code in the part of the if statement that evaluates to True. Note that in the for loop in this case we simply have process whose id is 0 start at iteration 0, then skip to 0 + numProcesses for its next iteration, and so on. Similarly, process 1 starts at iteration 1, skipping next to 1+ numProcesses, and continuing until REPs is reached. Each process performs similar single 'slices' or 'chunks of size 1' of the whole loop.


In [ ]:
%%writefile 03parallelLoopChunksOf1.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 8

    if (numProcesses <= REPS):

        for i in range(id, REPS, numProcesses):
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i))

    else:
        # can't have more processes than work; one process reports the error
        if id == 0 :
            print("Please run with number of processes less than \
or equal to {}.".format(REPS))

########## Run the main function
main()


Overwriting 03parallelLoopChunksOf1.py


In [ ]:
! mpirun --allow-run-as-root -np 4 python 03parallelLoopChunksOf1.py

On aa7fe3e77f51: Process 0 is performing iteration 0
On aa7fe3e77f51: Process 0 is performing iteration 4
On aa7fe3e77f51: Process 3 is performing iteration 3
On aa7fe3e77f51: Process 3 is performing iteration 7
On aa7fe3e77f51: Process 2 is performing iteration 2
On aa7fe3e77f51: Process 2 is performing iteration 6
On aa7fe3e77f51: Process 1 is performing iteration 1
On aa7fe3e77f51: Process 1 is performing iteration 5


### Exercises
- Run, using these numbers of processes, N: 1, 2, 4, and 8
- Compare source code to output.
- Change REPS to 16, save, rerun, varying N again.
- Explain how this pattern divides the iterations of the loop among the processes.

Which of the following is the correct assignment of loop iterations to processes for this code, when REPS is 8 and numProcesses is 4?


![picture](https://drive.google.com/uc?id=1eUsjxYdWXWqThO_rdLO91HaLqBLAAh_S)

# Point to point communication: the message passing pattern

The fundamental basis of coordination between independent processes is point-to-point communication between processes through the communication links in the MPI.COMM_WORLD. The form of communication is called message passing, where one process **sends** data to another one, who in turn must **receive** it from the sender. This is illustrated as follows:

![picture](https://drive.google.com/uc?id=1WJcOXq6Dn5TKF9Lng8r18_y2b23tHFe8)

## Message Passing Pattern: Key Problem

The following code represents a common error that many programmers have inadvertently placed in their code. The concept behind this program is that we wish to use communication between pairs of processes, like this:

![picture](https://drive.google.com/uc?id=1UJ2acj6XzphD2W6gnutNF2YGp6wU529Z)

For message passing to work between a pair of processes, one must send and the other must receive. If we wish to **exchange** data, then each process will need to perform both a send and a receive.
The idea is that process 0 will send data to process 1, who will receive it from process 0. Process 1 will also send some data to process 0, who will receive it from process 1. Similarly, processes 2 and 3 will exchange messages: process 2 will send data to process 3, who will receive it from process 2. Process 3 will also send some data to process 2, who will receive it from process 3.

If we have more processes, we still want to pair up processes together to exchange messages. The mechanism for doing this is to know your process id. If your id is odd (1, 3 in the above diagram), you will send and receive from your neighbor whose id is id - 1. If your id is even (0, 2), you will send and receive from your neighbor whose id is id + 1. This should work even if we add more than 4 processes, as long as the number of processes is divisible by 2.

![warning sign](https://drive.google.com/uc?id=1SEqDBTBSKwNVXzn-zueWa7fBCm5b1_MB)
**Warning** There is a problem with the following code called *deadlock*. This happens when every process is waiting on an action from another process. The program cannot complete. **To stop the program, choose the small square that appears after you choose to run the mpirun cell.**


In [ ]:
%%writefile 04messagePssingDeadlock.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id # 0
        if odd(id):
            #odd processes receive from their paired 'neighbor', then send
            receivedValue = comm.recv(source=id-1)
            comm.send(sendValue, dest=id-1)
        else :
            #even processes receive from their paired 'neighbor', then send
            receivedValue = comm.recv(source=id+1) # id=1
            comm.send(sendValue, dest=id+1) # 0, dest=1

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendValue, receivedValue)) # 0,2,rtwww, 0, 1

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()


Overwriting 04messagePssingDeadlock.py


In [ ]:
! mpirun --allow-run-as-root -np 2 python 04messagePssingDeadlock.py

![warning sign](https://drive.google.com/uc?id=1SEqDBTBSKwNVXzn-zueWa7fBCm5b1_MB)Remember,**To stop the program, choose the small square that appears after you choose to run the mpirun cell.**

#### What causes the deadlock?

Each process, regardless of its id, will execute a receive request first. In this model, recv is a **blocking** function- it will not continue until it gets data from a send. So every process is blocked waiting to receive a message.

#### Can you think of how to fix this problem?

Since recv is a **blocking** function, we need to have some processes send first, while others correspondingly recv first from those who send first. This provides coordinated exchanges.

Go to the next example to see the solution.


## Message Passing Patterns: avoiding deadlock

Let's look at a few more correct message passing examples.

### Fix the Deadlock

To fix deadlock of the previous example, we coordinate the communication between pairs of processes so that there is an ordering of sends and receives between them.

![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s)**Important:** The new code corrects deadlock with a simple change: odd process sends first, even process receives first. *This is the proper pattern for exchanging data between pairs of processes.*

In [ ]:
%%writefile 05messagePassing.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        sendValue = id
        if odd(id):
            #odd processes send to their paired 'neighbor', then receive from
            comm.send(sendValue, dest=id-1)
            receivedValue = comm.recv(source=id-1)
        else :
            #even processes receive from their paired 'neighbor', then send
            receivedValue = comm.recv(source=id+1)
            comm.send(sendValue, dest=id+1)

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendValue, receivedValue))

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()


Overwriting 05messagePassing.py


In [ ]:
! mpirun --allow-run-as-root -np 2 python 05messagePassing.py

Process 0 of 2 on aa7fe3e77f51 computed 0 and received 1
Process 1 of 2 on aa7fe3e77f51 computed 1 and received 0


# Nueva sección